In [1]:
## Import

from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import cv2
import os

from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical

from datetime import datetime

In [48]:
## Helper functions

def read_from_folder(directory, input_shape):
    input = np.empty(input_shape, dtype='uint8')
    image_count = 0

    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if os.path.isfile(f) and (f.endswith('.jpg') or f.endswith('.jpeg')):
            img = Image.open(f)
    
            # Resize
            size = 176, 100
            img = img.resize(size, Image.LANCZOS)
    
            numpydata = np.array(img)
            input[image_count] = numpydata
    
            image_count += 1
    
            if image_count == 37:
                break  # Stop processing if 37 images are reached
    
    while image_count < 37:
        input[image_count] = input[image_count - 1]
        image_count += 1
    
    return input

def get_csv_dict(directory, labels): ### IMPORTANT TO REMEMBER: This is for the limited csv, you MUST change this for the full version.
    data = pd.read_csv(directory)
    data_dict = {i: [] for i in labels}
    desired_rows = 1500
    
    for label in labels:
        filtered_data = data[data['label_id'].isin([label])] #### TODO: Make everything 1500 length

        if len(filtered_data) >= desired_rows:
            data_dict[label] =  filtered_data['video_id'].values[:desired_rows]
            continue
    
        rows_to_add = desired_rows - len(filtered_data)
        random_rows = filtered_data.sample(n=rows_to_add, replace=True)
        data_dict[label] = pd.concat([filtered_data, random_rows], ignore_index=True)['video_id'].values

    return data_dict
        
def generate_data_labels(labels, label, length):
    data_label = []
    for i in range(length):
        result = [1 if labels[j] == label else 0 for j in range(len(labels))]
        data_label.append(result)
    return np.array(data_label)

def get_batch(directory, data_dict, chosen_labels, label, amount, index):    
    data = np.array([])

    for i in range(index, (index+amount)):
        example = read_from_folder(f"{directory}/{data_dict[label][i]}", (37, 100, 176, 3))
        example = np.expand_dims(example, axis=0)
        data = append_or_copy_array(data, example)
    
    #data = read_from_file(data_directory + "data" + "_batch_" + str(batch_num) + ".npy")
    data_labels = generate_data_labels(chosen_labels, label, len(data))
    
    index += amount
    return data, data_labels, index

def append_or_copy_array(base_array, new_array):
    if base_array.size == 0:
        result_array = new_array
    else:
        result_array = np.concatenate((base_array, new_array), axis=0)

    return result_array

def load_model_names(log_file):
    model_names = set()

    if os.path.exists(log_file):
        with open(log_file, 'r') as file:
            model_names = set(file.read().splitlines())

    return model_names

def cleanup_models(save_dir, log_file, model_names):
    while len(model_names) > 10:
        oldest_model = min(model_names)
        oldest_model_path = os.path.join(save_dir, oldest_model)

        # Delete the oldest model file
        if os.path.exists(oldest_model_path):
            os.remove(oldest_model_path)

        # Remove the oldest model name from the set and the log file
        model_names.remove(oldest_model)
        with open(log_file, 'w') as file:
            file.write('\n'.join(model_names) + '\n')

def save_model(model, save_dir, log_file, model_name_prefix='model'):
    # Generate a unique model name
    timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
    model_name = f'{model_name_prefix}_{timestamp}.keras'

    # Save the model
    model_path = os.path.join(save_dir, model_name)
    model.save(model_path)

    # Update the model names set
    model_names = load_model_names(log_file)
    model_names.add(model_name)

    # Update the log file with the new model name
    with open(log_file, 'a') as file:
        file.write(model_name + '\n')

    # Remove oldest model if exceeding the maximum allowed models
    cleanup_models(save_dir, log_file, model_names)

    print(f'Model saved: {model_name}')

def load_model(model_path):
    return tf.keras.models.load_model(model_path)

def load_model_from_dir(save_dir, model):
    saved_models = [f for f in os.listdir(save_dir) if f.startswith('model_')]

    if not saved_models:
        print('No saved models found.')
        return model

    print('Saved Models:')
    for i, model_name in enumerate(saved_models, start=1):
        print(f'{i}. {model_name}')

    choice = input('Do you want to load a model? (y/n): ').lower()

    if choice == 'y':
        model_choice = int(input('Enter the number of the model to load: '))
        if 1 <= model_choice <= len(saved_models):
            selected_model = saved_models[model_choice - 1]
            model_path = os.path.join(save_dir, selected_model)
            loaded_model = load_model(model_path)
            print(f'Model {selected_model} loaded successfully.')
            return loaded_model
        else:
            print('Invalid choice. No model loaded.')
            return model

In [41]:
## Model architecture

model = tf.keras.Sequential([
    tf.keras.layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=(37, 100, 176, 3)),
    tf.keras.layers.MaxPooling3D((2, 2, 2)),
    tf.keras.layers.Dropout(0.25),  # Add dropout with a rate of 25%
    tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu'),
    tf.keras.layers.MaxPooling3D((2, 2, 2)),
    tf.keras.layers.Dropout(0.25),  # Add another dropout layer with the same rate
    tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [50]:
## Traning on batch

data_directory = "D:\\Magshimim\\Dataset\\20bn-jester-v1"
data_csv = "D:\\Magshimim\\Train.csv"
val_csv = "D:\\Magshimim\\Validation.csv"
chosen_labels = [14, 15]
log_file = "model_log.txt"
model_saves = "model checkpoints"

# Define loss function and optimizer
loss_fn = 'categorical_crossentropy'
optimizer = tf.keras.optimizers.Adam()

# Training loop using train_on_batch
num_epochs = 10
batch_size = 30  # Adjust batch size as needed
print("starting...")

# Compiling the model
model.compile(loss=loss_fn, optimizer=optimizer)

data_dict = get_csv_dict(data_csv, chosen_labels)
val_dict = get_csv_dict(val_csv, chosen_labels)

load_model_from_dir(model_saves, model)

accuracy_list = np.array([])
val_accuracy_list = np.array([])

for batch in range(int(len(data_dict[chosen_labels[0]]) / batch_size)):
    data = np.array([])
    data_labels = np.array([])
    test = np.array([])
    test_labels = np.array([])

    data_index = 0
    val_index = 0
    
    for label in chosen_labels:
        data_batch, data_labels_batch, data_index = get_batch(data_directory, data_dict, chosen_labels, label, batch_size, data_index)
        test_batch, test_labels_batch, val_inedx = get_batch(data_directory, val_dict, chosen_labels, label, int(batch_size / 10), val_index)
        
        data = append_or_copy_array(data, data_batch)
        data_labels = append_or_copy_array(data_labels, data_labels_batch)
        test = append_or_copy_array(test, test_batch)
        test_labels = append_or_copy_array(test_labels, test_labels_batch)
        
    data = tf.convert_to_tensor(data)
    data_labels = tf.convert_to_tensor(data_labels)
    test = tf.convert_to_tensor(test)
    test_labels = tf.convert_to_tensor(test_labels)
    
    print(data.shape, data_labels.shape, test.shape, test_labels.shape)
    
    for epoch in range(num_epochs):
        total_loss = 0
        correct_predictions = 0

        for i in range(0, len(data), batch_size):
            print(i)
            batch_data = data[i:i+batch_size]
            batch_labels = data_labels[i:i+batch_size]

            with tf.GradientTape() as tape:
                predictions = model(batch_data)
                if loss_fn == 'categorical_crossentropy':
                    loss = tf.keras.losses.categorical_crossentropy(batch_labels, predictions)
                else:
                    raise ValueError("Invalid loss function specified.")

            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

            total_loss += loss.numpy() 
            correct_predictions += np.sum(np.argmax(predictions, axis=1) == np.argmax(batch_labels, axis=1))

        average_loss = total_loss / (len(data) / batch_size)
        accuracy = correct_predictions / len(data)

        # Validation
        val_predictions = model(test)
        val_loss = tf.keras.losses.categorical_crossentropy(test_labels, val_predictions).numpy()
        val_accuracy = np.sum(np.argmax(val_predictions, axis=1) == np.argmax(test_labels, axis=1)) / len(test)

        save_model(model, model_saves, log_file)

        accuracy_list = np.append(accuracy_list, accuracy)
        val_accuracy_list = np.append(val_accuracy_list, val_accuracy)
        print(f'Epoch {epoch+1}, Loss: {average_loss}, Accuracy: {accuracy}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}')

starting...
Saved Models:
1. model_20231228140503.keras
2. model_20231228140530.keras
3. model_20231228140618.keras


Do you want to load a model? (y/n):  y
Enter the number of the model to load:  1


Model model_20231228140503.keras loaded successfully.
(60, 37, 100, 176, 3) (60, 2) (6, 37, 100, 176, 3) (6, 2)
0



KeyboardInterrupt



In [ ]:
## Saving the model 
model.save('Model_test.keras')

In [ ]:
## Graph the accuracy
# Plot the data
plt.plot(accuracy_list)
plt.plot(val_accuracy_list)

# Add labels and title
plt.xlabel('Accuracy')
plt.ylabel('Epoch')
plt.title('Modle Accuracy Plot')

# Show the plot
plt.show()